# TOOL MCGVYERING: TOOL CREATION WITH AUXILARY OBJECTS


## Required Libraries & Parameters

In [1]:
import sys
sys.path.append('../src')
import numpy as np
import robotic as ry

from RAI import RAI
from SEG import SEG
from SQF import SQF
from Score import Score

cam_list = ["cam_front_left", "cam_front_right", "cam_back_right", "cam_back_left", "cam_up", "cam_front", "cam_left", "cam_back", "cam_right"] 
filter = 1
model_path = "../src/models/tools/simple/parts"
target_path = "../src/point_clouds_ref/"
scene = ry.Config()
scene.addFile("../src/config/tools_simple2_s.g") 
ref_file = "../src/parameters_ref.csv"
cand_file = "../src/parameters_cand.csv"
reference_tool = "rake" # shovel, hammer, rake # Select a reference tool

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## STEP 1: Generate the point cloud for the reference tool

In [2]:
print("Generating the reference point cloud")
rai = RAI(verbose=0)
segments_ref = rai.generate_ref_point_cloud(model_path, cam_list, filter = filter, object_name=reference_tool)

Generating the reference point cloud
Model name: rake_head
Model name: rake_handle


## STEP 2: Generate the point cloud for the candidate objects

In [3]:
print("Generating the candidate point cloud")
ptc_cand, _, _ = rai.get_raw_point_cloud(scene, cam_list[0:7], filter = filter)

Generating the candidate point cloud


## STEP 3: Remove the plane from the scene

In [4]:
print("Removing the plane from the scene")
seg = SEG(verbose=0)
pcl_filtered = seg.RANSAC_plane(ptc_cand)

print("Segmenting the objects")
segments_cand = seg.segment_objects(pcl_filtered, scene)

Removing the plane from the scene
Segmenting the objects


# STEP Alternative: You can also use the premade object dataset (PS: Dont run Steps 2,3,6)

In [8]:
segments_cand = RAI.read_point_cloud_from_folder("../src/models/dataset/")

## STEP 4: Fit SuperQuadric function to the objects

In [5]:
param_ref_full = SQF.load_parameters_from_csv(ref_file)
param_cand_full = SQF.load_parameters_from_csv(cand_file)

param_ref = {key: param_ref_full[key] for key in segments_ref.keys() if key in param_ref_full}
param_cand = {key: param_cand_full[key] for key in segments_cand.keys() if key in param_cand_full}

missing_ref_keys = set(segments_ref.keys()) - set(param_ref.keys())
if missing_ref_keys:
    print(f"Fitting SuperQuadric function for missing reference objects: {missing_ref_keys}")
    for key in missing_ref_keys:
        pcd = segments_ref[key]
        sqf = SQF(pcd, verbose=0)
        try:
            param, _, _, _ = sqf.fit(sq_type=[0])
            if len(param) < 12:
                param = np.pad(param, (0, 12 - len(param)), mode='constant', constant_values=0)
            param_ref[key] = param
        except ValueError as e:
            print(f"Error fitting SQF for reference object {key}: {e}")
    SQF.save_parameters_to_csv({**param_ref_full, **param_ref}, ref_file)

missing_cand_keys = set(segments_cand.keys()) - set(param_cand.keys())
if missing_cand_keys:
    print(f"Fitting SuperQuadric function for missing candidate objects: {missing_cand_keys}")
    for key in missing_cand_keys:
        pcd = segments_cand[key]
        sqf = SQF(pcd, verbose=0)
        try:
            param, _, _, _ = sqf.fit(sq_type=[0])
            if len(param) < 12:
                param = np.pad(param, (0, 12 - len(param)), mode='constant', constant_values=0)
            param_cand[key] = param
        except ValueError as e:
            print(f"Error fitting SQF for candidate object {key}: {e}")
    SQF.save_parameters_to_csv({**param_cand_full, **param_cand}, cand_file)


Parameters loaded from ../src/parameters_ref.csv
Parameters loaded from ../src/parameters_cand.csv
Fitting SuperQuadric function for missing candidate objects: {'obj_rectangle', 'obj_square', 'obj_comb'}
Parameters saved to ../src/parameters_cand.csv


## STEP 5: Find the best matching candidates objects to the reference tool set

In [6]:
print("Finding the best matching candidate objects to the reference tool set")
score = Score(param_ref, param_cand, verbose=1)
match_1, match_2 = score.algorithm_1()

Finding the best matching candidate objects to the reference tool set
Matches: [('rake_head', 'obj_prism_short'), ('rake_handle', 'obj_prism_long')], Total Error: 1.0970, Shape Error: 0.4170, Size Error: 0.3502, Ratio Error: 0.6595
Matches: [('rake_head', 'obj_prism_long'), ('rake_handle', 'obj_prism_short')], Total Error: 1.6883, Shape Error: 0.4170, Size Error: 0.7587, Ratio Error: 1.0250
Matches: [('rake_head', 'obj_comb'), ('rake_handle', 'obj_prism_long')], Total Error: 4.8872, Shape Error: 0.2453, Size Error: 0.9679, Ratio Error: 7.3480
Matches: [('rake_head', 'obj_comb'), ('rake_handle', 'obj_prism_short')], Total Error: 5.2367, Shape Error: 0.2453, Size Error: 1.2198, Ratio Error: 7.5431
Matches: [('rake_head', 'obj_square'), ('rake_handle', 'obj_prism_long')], Total Error: 5.2725, Shape Error: 0.1454, Size Error: 0.9915, Ratio Error: 8.2713
Matches: [('rake_head', 'obj_square'), ('rake_handle', 'obj_prism_short')], Total Error: 5.6219, Shape Error: 0.1454, Size Error: 1.2433, 

## STEP 6: Create the tool

In [ ]:
rai.create_tool(scene, match_1, match_2)